In [2]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-vz0evzzk
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-vz0evzzk
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


In [4]:
from dataloader_places import PlacesDataset
from torch.utils.data import DataLoader
import clip
import torch
from tqdm import tqdm

idx_to_classname ={0: 'bathroom', 1: 'bedroom', 2: 'childs_room',
                   3: 'classroom', 4: 'dressing_room', 5: 'living_room',
                   6: 'studio', 7: "swimming_pool"}

data_path = "../adversarial-sets/data/Places8_paths_and_labels_complete_train.npy"

places_ds = [PlacesDataset(data_path,
                           onlylabels=[k]) for k in range(8)]
batch_size = 64
train_dataloaders_class = {k: DataLoader(places_ds[k],
                                         batch_size=batch_size,
                                         shuffle=False,
                                         num_workers=8) for k in range(8)}
for k in range(8):
    print(f"\nDataloader: {batch_size} batch size | {len(train_dataloaders_class[k])} batches | {len(train_dataloaders_class[k].dataset)} images")

print(clip.available_models())
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)


Dataloader: 1 batch size | 9991 batches | 9991 images
['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']
cuda


In [5]:
model, preprocess = clip.load("ViT-B/32", device=device)
print(model)

 38%|██████████████▊                        | 128M/338M [00:24<00:40, 5.48MiB/s]


KeyboardInterrupt: 

In [ ]:
print(preprocess)

In [42]:
for i in range(8):
    image_features = []
    text_features = []
    for inputs, labels, _ in tqdm(train_dataloaders_class[i]):
        inputs = inputs.to(device)
        text_inputs = torch.cat([clip.tokenize(
            f"a photo of a {idx_to_classname[label.item()]}", truncate=True) for label in labels]).to(device)

        # Calculate features
        with torch.no_grad():
            img_feat_batch = model.encode_image(inputs).cpu().data.numpy()
            text_feat_batch = model.encode_text(text_inputs).cpu().data.numpy()
            for idx, _ in enumerate(img_feat_batch):
                image_features.append(img_feat_batch[idx].tolist())
                text_features.append(text_feat_batch[idx].tolist())
    
    image_features = np.asarray(image_features)
    text_features = np.asarray(text_features)
    np.save(f"places8_image_features_clip_class_{i}.npy", image_features)
    np.save(f"places8_text_features_clip_class_{i}.npy", text_features)


100%|████████████████████████████████████████████████████████████████| 5701/5701 [43:10<00:00,  2.20it/s]


In [45]:
print(len(image_features))

364806

In [46]:
import numpy as np

image_features = np.asarray(image_features)
text_features = np.asarray(text_features)
np.save("places8_image_features_clip.npy", image_features)
np.save("places8_text_features_clip.npy", text_features)